# Data Exploration

## Background 

### Data Source
The data used in this notebook is sourced from the National Centers for Environmental Information (NCEI): [Global Historical Climatology Network (GHCN) - Hourly](https://www.ncei.noaa.gov/products/global-historical-climatology-network-hourly). Refer to their documentation and terms of use.


#### Data Set

Station_ID: the station identification code. The first two characters signify the FIPS country code, the third character is a network code identifying the station numbering system used, and the remaining eight characters contain the actual station ID.

Station_Name: the name of the station.

Year: the year the observation was taken in Coordinated Universal Time (UTC).

Month: the month the observation was taken in Coordinated Universal Time (UTC).

Day: the day the observation was taken in Coordinated Universal Time (UTC).

Hour: the hour the observation was taken in Coordinated Universal Time (UTC).

Latitude: latitude of the station (in decimal degrees). North (+); South (-).

Longitude: the longitude of the station (in decimal degrees). East (+); West (-).

Temperature: 2 meter (circa) Above Ground Level Air (dry bulb) Temperature (⁰C to tenths)


Notes: 
- Raw data was removed in download_ghcn.py for storage purposes.
- GHCN hourly dataset contained psv files for individual stations in specific years. When processing the data, it was converted to csv format files for all California stations in years 2003 - 2023.
- Most columns were dropped as they were not needed. Columns kept were described above.
- Duplicate rows that had completely same column values were dropped.

# Data Cleaning

### Set up

In [ ]:
import pandas as pd # type: ignore
import numpy as np
import sys
import os

# Update paths to get source code from notebook_utils
curr_dir = os.path.dirname(os.path.abspath('notebooks'))
proj_dir = os.path.dirname(curr_dir)
src_path = os.path.join(proj_dir, 'src')
sys.path.append(src_path)

from notebook_utils.preprocessing import *

CA_stations = get_reduced_df()

### Data Examination

In [ ]:
CA_stations.head()

In [ ]:
CA_stations.info()

In [ ]:
CA_stations.describe()

The temperature column has a really high max celsius value which is 902 degrees celsius. This is unreasonably high. After doing some searching, we found that the highest recorded temperature value was 56.7 degrees celsius in California 1913. 

There is also an unreasonably low temperature observation of -99 degrees celsius since the lowest recorded temperature observation on Earth was -98 degrees in Antartica. 

# Cleaning Invalid Data

- Handle Missing Values  (e.g., mean/median impuation, interpolation, forward or backward fill, k-nearest neighbors imputation, deletion)
- Handle Outliers  (e.g., visual inspection by boxplots, Z-score and IQR method, or data transformation by log transformation and winsorization)
- Handle inconsistencies (e.g., checking ranges to ensure temperature values fall within a reasonable range, unit consistency, string matching and standardization), and duplicates (identify and remove duplicates) in the dataset

Notes:
- For non-leap years, there should be 8760 rows (for each hour) for each station.
- For leap years, there should be 8784 rows (for each hour) for each station
- Leap years from 2003-2023 include: 2004, 2008, 2012, 2016, and 2020
- The reduced files contain 99 CA stations.
- Some stations are not observed each year from 2003-2023.


## Handling Inconsistencies

1. Ensure temperature observations are within -50°C and 60°C
2. Temperature values above the reasonable range will be converted to NaN.

In [ ]:
# Check how many rows are below -50 and above 60 degrees celsius
CA_stations[(CA_stations['temperature'] < -50) | (CA_stations['temperature'] > 60)]

In [ ]:
# Write code to turn temperature outside of the specified range into NaN
CA_stations.loc[(CA_stations['temperature'] < -50) | (CA_stations['temperature'] > 60), 'temperature'] = np.nan

## Deleting Duplicate Rows

There are rows with the same value in each column except temperature. In these cases we will average out the temperature observations and delete the extra rows.

In [ ]:
grouped_df = CA_stations.groupby(['Station_ID', 'Station_name', 'Year', 'Month', 'Day', 'Hour', 'Latitude', 'Longitude']).agg({'temperature': 'mean'}).reset_index()
CA_stations = grouped_df

## Handling Missing Values

### Filling missing rows

In [ ]:
# create a reference dataframe with all stations with hours from 2003 to 2023
full_df = create_full_df()

In [ ]:
full_df.describe()

In [ ]:
# Fill in completely missing rows
missing_rows = find_missing_rows(full_df, CA_stations)
CA_stations = add_missing_rows(CA_stations, missing_rows)

In [ ]:
error = check_station_rows(CA_stations)
print(error)

In [ ]:
CA_stations.head()

### Station Deletion

Stations with no Station_name values will be dropped as that means there are no observations recorded for them at all from 2003 - 2023
- This is because the above Station_name for missing rows were filled in by using the Station_name used from a filled column with the same Station_ID.

In [ ]:
CA_stations.dropna(subset=['Station_name'], inplace=True)   # 21 stations dropped

## Filling missing temperature values

In [ ]:
null_count = CA_stations.isna().sum()
null_percent = (null_count/CA_stations.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

### Missing Temperature Values: Handling large gaps in temperature observations

We define a "large gap" as being a gap in the temperature column that is more than a day.

Large gaps in temperature observations will be handled via interpolation

#### Cubic Spline Interpolation for large gaps of missing temperature values

In [ ]:
from notebook_utils.preprocessing import cubic_spline_interpolate

CA_interpolated_df = cubic_spline_interpolate(CA_stations,gap_hours=24)

In [ ]:
CA_interpolated_df.head()

In [ ]:
null_count = CA_interpolated_df.isna().sum()
null_percent = (null_count/CA_interpolated_df.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

### Missing Temperature Values: Handling short gaps in temperature observations

We define a "short gap" as being a gap in the temperature column that is less than a day. 

Short gaps in temperature observations will be handled with forward/backward fill.

In [ ]:
final_df = fill_gaps(CA_interpolated_df)

In [ ]:
null_count = final_df.isna().sum()
null_percent = (null_count/final_df.shape[0]) * 100
print(f'Percentage of null values for each column: \n{null_percent}')

#### Validation

In [ ]:
import matplotlib.pyplot as plt

# make copies to not alter original dataframes
plot_CA_stations = CA_stations.copy()
plot_CA_interpolated = CA_interpolated_df.copy()

plot_CA_stations['Datetime'] = pd.to_datetime(plot_CA_stations[['Year', 'Month', 'Day', 'Hour']])
plot_CA_interpolated['Datetime'] = pd.to_datetime(plot_CA_interpolated[['Year', 'Month', 'Day', 'Hour']])

plot_CA_stations.set_index('Datetime', inplace=True)
plot_CA_interpolated.set_index('Datetime', inplace=True)

monthly_avg_original = plot_CA_stations['temperature'].resample('ME').mean()
monthly_avg_interpolated = plot_CA_interpolated['temperature'].resample('ME').mean()

plt.figure(figsize=(14, 7))

plt.plot(monthly_avg_original.index, monthly_avg_original.values, 'o-', label='Original Data', markersize=4)

plt.plot(monthly_avg_interpolated.index, monthly_avg_interpolated.values, 'x-', label='Interpolated Data', markersize=4)

plt.xlabel('Datetime')
plt.ylabel('Temperature (°C)')
plt.title('Monthly Average Temperature (2003-2023)')
plt.legend()
plt.grid(True)
plt.show()

## Handling Outliers

Outliers can skew our statistical analysis of the data. 

### 1. Visual Inspection

#### Boxplot

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.boxplot(x='Year', y='temperature', data=CA_interpolated_df)
plt.title('Temperature Boxplot by Year')
plt.xticks(rotation=90)  
plt.ylabel('Temperature')
plt.show()

### 2. Statistical Methods

#### Z-Score

In [ ]:
from scipy import stats

# z_stations dataframe will have a Z_score column added to it
z_stations = CA_interpolated_df.copy()
z_stations['Z_score'] = stats.zscore(z_stations['temperature'])
z_thresh = 3

# calculate outliers using Z-score
z_outliers = z_stations[(z_stations['Z_score'] < -z_thresh) | (z_stations['Z_score'] > z_thresh)]
total_observations = z_stations.shape[0]

num_outliers = z_outliers.shape[0]

percent_outliers = (num_outliers / total_observations) * 100

print(f'{percent_outliers:.2f}% of the observations are outliers')

### 3. Drop Outliers

In [ ]:
z_stations_cleaned = z_stations[~((z_stations['Z_score'] < -z_thresh) | (z_stations['Z_score'] > z_thresh))]

# Drop the Z_score column as it's no longer needed
z_stations_cleaned = z_stations_cleaned.drop(columns=['Z_score'])

# The z_stations_cleaned dataframe now contains the data without the outliers
z_stations_cleaned.describe()

### UTC to Local Time Conversion

In [ ]:
z_stations_cleaned['Datetime'] = pd.to_datetime(z_stations_cleaned[['Year', 'Month', 'Day', 'Hour']])
z_stations_cleaned = z_stations_cleaned.drop(columns=['Year', 'Month', 'Day', 'Hour'])

z_stations_cleaned['Datetime_local'] = z_stations_cleaned['Datetime'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
z_stations_cleaned['Year'] = z_stations_cleaned['Datetime_local'].dt.year
z_stations_cleaned['Month'] = z_stations_cleaned['Datetime_local'].dt.month
z_stations_cleaned['Day'] = z_stations_cleaned['Datetime_local'].dt.day
z_stations_cleaned['Hour'] = z_stations_cleaned['Datetime_local'].dt.hour
z_stations_cleaned = z_stations_cleaned.drop(columns=['Datetime', 'Datetime_local'])

# remove 2002 observations
z_stations_cleaned = z_stations_cleaned[z_stations_cleaned['Year'] != 2002]

# rearrange columns
cols = ['Station_ID','Station_name', 'Latitude', 'Longitude','Year','Month','Day','Hour','temperature']
z_stations_cleaned = z_stations_cleaned[cols]

In [ ]:
z_stations_cleaned.head()   

### Write dataframe to csv files by year

In [ ]:
output_folder = '../data/processed/ghcn_clean'
os.makedirs(output_folder, exist_ok=True)

years = z_stations_cleaned['Year'].unique()

# Capitalize the temperature column to match the rest.
z_stations_cleaned = z_stations_cleaned.rename(columns={'temperature': 'Temperature'})
z_stations_cleaned.sort_values(by=['Station_ID', 'Year', 'Month', 'Day', 'Hour'], inplace=True)

for year in years:
    yearly_data = z_stations_cleaned[z_stations_cleaned['Year'] == year]
    output_file = os.path.join(output_folder, f'CA_{year}_clean.csv')
    yearly_data.to_csv(output_file, index=False)

print('New datafrane yearly files saved to ghcn_cleaned')